# <b>spacemake</b> - Automated Analysis

In [ ]:
import numpy as np
import scanpy as sc
import spacemake as smk
from spacemake.spatial import util as spatial_util
from spacemake.report import utils as report_utils
import matplotlib.pyplot as plt

from functools import partial

In [ ]:
import os

# limit the resources used by scanpy
os.environ['OMP_NUM_THREADS'] = '1'   # For OpenMP
os.environ['OPENBLAS_NUM_THREADS'] = '1'  # For OpenBLAS
os.environ['MKL_NUM_THREADS'] = '1'  # For MKL (Intel Math Kernel Library)
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'  # For macOS Accelerate
os.environ['NUMEXPR_NUM_THREADS'] = '1'  # For NumExpr

### Loading and cleaning-up the data

In [ ]:
# Parameters - this cell will be replaced by papermill
adata_path = "path/to/adata.h5ad"
umi_cutoff = 100
clustering_resolutions = [0.4, 0.6, 0.8, 1.0, 1.2]
detect_tissue = True
adata_output = "path/to/output.h5ad"
is_spatial = True

N_CELLS_MIN = 100  # minimum number of cells to proceed with analysis
N_GENES_MIN = 1000  # minimum number of genes to proceed with analysis

In [ ]:
adata = sc.read_h5ad(adata_path)

if 'spatial' in adata.obsm.keys() and detect_tissue:
    adata = spatial_util(adata, umi_cutoff)
else:
    adata = adata[adata.obs.total_counts > umi_cutoff, :]

adata.obs_names_make_unique()
adata.var_names_make_unique()
adata.raw = adata

ncells, ngenes = adata.shape

if ncells < N_CELLS_MIN or ngenes <= N_GENES_MIN:
    adata.write(adata_output)
    print("Notebook cannot continue because there are too few cells or too few genes!")
    import sys
    sys.exit(0)

## Overview of sample metrics

In [ ]:
metrics_table_df = report_utils.create_metrics_table_df(adata, umi_cutoff)

In [ ]:
visualizer = smk.pl.TabVisualizer()

visualizer.add_plot_group(
        smk.pl.PlotGroup(
        name=f"Summary Beads",
        description=f"QC tables for the sample",
        plots=[smk.pl.DataFrameTable(
            title="Summary Beads",
            description="[Description]",
            data=metrics_table_df
        )]
    )
)

In [ ]:
display(visualizer.generate_html())

## Histogram of sample metrics

In [ ]:
# Set as multi histogram function?
metrics = {'n_reads': '# of reads',
           'reads_per_counts': '# of reads/UMI',
           'n_genes_by_counts': '# of genes',
           'total_counts': '# of UMIs',
           'pct_counts_mt': '% of mitochondrial counts'}

metrics_colors = {
    "total_counts": "#E69F00",
    "n_genes_by_counts": "#56B4E9",
    "n_reads": "#009E73",
    "reads_per_counts": "#CC79A7",
    "pct_counts_mt": "black",
}

fig, axes = plt.subplots(3, 2, figsize=(7, 6))
for i, (metric_key, metric_desc) in enumerate(metrics.items()):
    smk.pl.histogram(adata.obs[metric_key], axes.flatten()[i], 100, metrics_colors[metric_key])
    axes.flatten()[i].set_ylabel("# of\nspatial units")
    axes.flatten()[i].set_xlabel(metric_desc)

axes[2, 1].axis("off")
plt.tight_layout()

## Distribution of UMIs in 2D
- <b>Original data</b> = original UMI counts

- <b>Scaled data</b> = the top 10% of UMIs (ordered high to low) are set to the minimum UMI of the first 10% UMIs, for visualisation purposes.

### Original

In [ ]:
smk.pl.spatial(adata, color="total_counts")

### Scaled

In [ ]:
smk.pl.spatial(adata, color="total_counts", vmax=np.nanquantile(adata.obs["total_counts"], 0.9))

## Clustering and markers

### Highly variable genes and normalisation

In [ ]:
sc.pp.normalize_total(adata)
sc.pp.log1p(adata, base=2)
sc.pp.highly_variable_genes(adata, flavor='seurat', n_top_genes=2000)

### Dimensionality reduction and clustering

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
n_pcs = adata.uns['pca']['variance'].size
n_pcs = n_pcs if n_pcs < 40 else 40

sc.pp.neighbors(adata, n_pcs=n_pcs)
try:
    sc.tl.umap(adata)   
except TypeError:
    pass

### Clustering and spatial plotting

In [ ]:
for res in clustering_resolutions:
    try:
        res_key = 'leiden_' + str(res)
        
        sc.tl.leiden(adata, resolution = res, key_added = res_key)
        sc.tl.rank_genes_groups(adata, res_key, method='t-test', key_added = 'rank_genes_groups_' + res_key, pts=True,
            use_raw = False)
    except ZeroDivisionError as e:
        pass

### Saving the final object

In [ ]:
adata.write(adata_output)

### Visualization

In [ ]:
visualizer = smk.pl.TabVisualizer()
metric_desc = "Spatial visualization of clustering results"

for res in clustering_resolutions:
    plots = []
    # UMAP
    plot = smk.pl.Plot(
        title=metric_desc,
        description=f"Clusters in UMAP space for leiden resolution {res}",
        plot_func=partial(smk.pl.umap, adata, color=f'leiden_{res}')
    )
    plots.append(plot)

    if is_spatial:
        # Spatial clusters
        plot = smk.pl.Plot(
            title=metric_desc,
            description=f"Clusters in physical space for leiden resolution {res}",
            plot_func=partial(smk.pl.spatial, adata, color=f'leiden_{res}')
        )
        plots.append(plot)

    group = smk.pl.PlotGroup(
        name=f"Resolution: {res}",
        description=f"Analysis results for resolution {res}",
        plots=plots
    )
    
    visualizer.add_plot_group(group)

In [ ]:
display(visualizer.generate_html())

In [ ]:
# # [...] or, you can generate the plots yourself!

# resolution = clustering_resolutions[0]

# smk.pl.umap(adata, key=f"leiden_{resolution}")

# # Spatial
# smk.pl.spatial(adata, color=f"leiden_{resolution}")